# Proyek Analisis Data: Air Quality
- **Nama:** Rio Fardinan
- **Email:** riofardinan123@gmail.com
- **ID Dicoding:** Rio Fardinan

## Menentukan Pertanyaan Bisnis

- Bagaimana trend kualitas udara di stasiun guanyuan dari tahun ke tahun?
- faktor apa yang paling berpengaruh terhadap perubahan kualitas udara?
- Bagaimana trend curah hujan di stasiun guanyuan setiap bulannya?

## Import Semua Packages/Library yang Digunakan

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap

## Data Wrangling

### Gathering Data

In [ ]:
folder_path = 'data/'

air_quality_df = pd.read_csv(folder_path + "PRSA_Data_Guanyuan_20130301-20170228.csv")
air_quality_df.head()

**Insight:**
- Mengetahui bahwa data memiliki variabel dalam data.
- Mengetahui tipe data membantu untuk memilih teknik analisis dan pembersihan data yang sesuai.

### Assessing Data

In [ ]:
air_quality_df.info()

In [ ]:
air_quality_df.isna().sum()

In [ ]:
air_quality_df.duplicated().sum()

**Insight:**
- Mengetahui variabel mana yang memiliki nilai kosong
- Mengetahui apakah ada duplikasi data

### Cleaning Data

In [ ]:
air_quality_df['PM2.5'].fillna(value=air_quality_df['PM2.5'].mean(), inplace=True)
air_quality_df['PM10'].fillna(value=air_quality_df['PM10'].mean(), inplace=True)
air_quality_df['SO2'].fillna(value=air_quality_df['SO2'].mean(), inplace=True)
air_quality_df['NO2'].fillna(value=air_quality_df['NO2'].mean(), inplace=True)
air_quality_df['CO'].fillna(value=air_quality_df['CO'].mean(), inplace=True)
air_quality_df['O3'].fillna(value=air_quality_df['O3'].mean(), inplace=True)
air_quality_df['TEMP'].fillna(value=air_quality_df['TEMP'].mean(), inplace=True)
air_quality_df['PRES'].fillna(value=air_quality_df['PRES'].mean(), inplace=True)
air_quality_df['DEWP'].fillna(value=air_quality_df['DEWP'].mean(), inplace=True)
air_quality_df['RAIN'].fillna(value=air_quality_df['RAIN'].mean(), inplace=True)
air_quality_df['wd'].fillna(value=air_quality_df['wd'].mode()[0], inplace=True)
air_quality_df['WSPM'].fillna(value=air_quality_df['WSPM'].mean(), inplace=True)

air_quality_df['date'] = pd.to_datetime(air_quality_df[['year', 'month', 'day', 'hour']])
air_quality_df.set_index('date', inplace=True)

**Insight:**
- Mengatasi missing value dengan mengisi data menggunakan nilai rata-rata dan nilai modus

## Exploratory Data Analysis (EDA)

### Explore ...

In [ ]:
air_quality_df.describe(include="all")

In [ ]:
air_quality_df.corr(numeric_only = True)

In [ ]:
air_quality_df.groupby("year").mean(numeric_only=True)

In [ ]:
air_quality_df.groupby("month").mean(numeric_only=True)

In [ ]:
air_quality_df.groupby("day").mean(numeric_only=True)

In [ ]:
air_quality_df.groupby("hour").mean(numeric_only=True)

**Insight:**
- Mengetahui mean, standar deviasi, nilai minimum, maksimum, serta persentil (25%, 50%, dan 75%) dari data
- Mengetahui Korelasi Antar Variabel
- Mengetahui Trend Polusi Tahunan, Musiman, dan harian

## Visualization & Explanatory Analysis

### Pertanyaan 1: Bagaimana trend kualitas udara di stasiun guanyuan dari tahun ke tahun?

In [ ]:
monthly_avg_all = air_quality_df[['PM2.5', 'PM10', 'SO2', 'NO2', 'O3']].resample('M').mean()

plt.figure(figsize=(14, 8))

plt.plot(monthly_avg_all.index, monthly_avg_all['PM2.5'], label='PM2.5', color='orange')
plt.plot(monthly_avg_all.index, monthly_avg_all['PM10'], label='PM10', color='red')
plt.plot(monthly_avg_all.index, monthly_avg_all['SO2'], label='SO2', color='blue')
plt.plot(monthly_avg_all.index, monthly_avg_all['NO2'], label='NO2', color='purple')
plt.plot(monthly_avg_all.index, monthly_avg_all['O3'], label='O3', color='green')

plt.title('Trend Rata-Rata Kualitas Udara di Stasiun Guanyuan')
plt.xlabel('Tahun')
plt.ylabel('Konsentrasi (µg/m³)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
monthly_avg_all = air_quality_df[['CO']].resample('M').mean()

plt.figure(figsize=(14, 8))

plt.plot(monthly_avg_all.index, monthly_avg_all['CO'], label='CO', color='orange')

plt.title('Trend Rata-Rata Kualitas Udara di Stasiun Guanyuan')
plt.xlabel('Tahun')
plt.ylabel('Konsentrasi (µg/m³)')
plt.legend()
plt.grid(True)
plt.show()

### Pertanyaan 2: Faktor apa yang paling berpengaruh terhadap perubahan kualitas udara?

In [ ]:
corr_matrix = air_quality_df[['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)

plt.title('Heatmap Korelasi antara Polutan dan Faktor Cuaca')
plt.show()

### Pertanyaan 3: Bagaimana trend curah hujan di stasiun guanyuan setiap bulannya?

In [ ]:
if 'date' not in air_quality_df.columns:
    air_quality_df['date'] = pd.to_datetime(air_quality_df[['year', 'month', 'day', 'hour']])
    air_quality_df.set_index('date', inplace=True)

monthly_rainfall = air_quality_df.groupby([air_quality_df.index.year, air_quality_df.index.month])['RAIN'].mean().unstack(level=0)

plt.figure(figsize=(12, 6))

for year in monthly_rainfall.columns:
    plt.plot(monthly_rainfall.index, monthly_rainfall[year], marker='o', label=str(year))

plt.title('Trend Curah Hujan Bulanan di Stasiun Guanyuan (Tiap Tahun)')
plt.xlabel('Bulan')
plt.ylabel('Rata-rata Curah Hujan (mm)')
plt.xticks(ticks=range(1, 13), labels=['Jan', 'Feb', 'Mar', 'Apr', 'Mei', 'Jun', 'Jul', 'Agu', 'Sep', 'Okt', 'Nov', 'Des'])
plt.legend(title='Tahun')
plt.grid(True)
plt.tight_layout()
plt.show()

## Analisis Lanjutan (Opsional)

In [ ]:
stations_coordinates = {
    "Aotizhongxin": [40.0378, 116.4188],
    "Changping": [40.2181, 116.2215],
    "Dingling": [40.2413, 116.2343],
    "Guanyuan": [39.9298, 116.3655],
    "Huairou": [40.3243, 116.6318],
    "Nongzhanguan": [39.9336, 116.4667],
    "Shunyi": [40.1282, 116.6546],
    "Tiantan": [39.8869, 116.4129],
    "Wanliu": [39.9741, 116.3138],
    "Wanshouxigong": [39.8787, 116.3549],
}

station_data = []
for station, coords in stations_coordinates.items():
    avg_pm25 = air_quality_df['PM2.5'].mean()
    station_data.append((station, coords[0], coords[1], avg_pm25))

map_beijing = folium.Map(location=[39.9042, 116.4074], zoom_start=10)

for station, lat, lon, pm25 in station_data:
    folium.CircleMarker(
        location=[lat, lon],
        radius=pm25 / 10,
        popup=f"{station}: {pm25:.2f} µg/m³",
        color="red" if pm25 > 75 else "green",
        fill=True,
        fill_color="red" if pm25 > 75 else "green",
        fill_opacity=0.7,
    ).add_to(map_beijing)

heat_data = [[lat, lon, pm25] for _, lat, lon, pm25 in station_data]
HeatMap(heat_data).add_to(map_beijing)

display(map_beijing)

kita bisa membuat peta untuk melihat distribusi konsentrasi polutan di berbagai area.  Menggunakan peta panas (heatmap) untuk melihat bagaimana polutan seperti PM2.5 tersebar di Beijing. Dengan data dari beberapa stasiun, kita bisa membandingkan kualitas udara antar daerah atau kawasan di Beijing.

## Conclusion

- Bagaimana trend kualitas udara di stasiun guanyuan dari tahun ke tahun?
> Secara keseluruhan, terdapat pola fluktuasi yang signifikan dari tahun ke tahun pada berbagai polutan, terutama pada 2016 yang menunjukkan konsentrasi yang lebih tinggi dibanding tahun lainnya. Hal ini mungkin disebabkan oleh kondisi lingkungan, cuaca, atau aktivitas tertentu yang mempengaruhi kualitas udara di stasiun Guanyuan selama periode tersebut.
- Faktor apa yang paling berpengaruh terhadap perubahan kualitas udara?
> Suhu (TEMP) tampaknya menjadi faktor cuaca yang paling berpengaruh terhadap perubahan kualitas udara, khususnya karena pengaruhnya yang negatif terhadap polutan seperti PM2.5, PM10, NO2, dan CO, serta pengaruh positif terhadap O3. Namun, korelasi yang ditunjukkan tidak sepenuhnya kuat untuk menyimpulkan pengaruh langsung dan pasti, karena faktor cuaca lain dan variabel lingkungan lain juga mungkin berperan.
- Bagaimana trend curah hujan di stasiun guanyuan setiap bulannya?
>  Periode musim hujan umumnya terjadi dari Mei hingga Agustus, dengan curah hujan yang meningkat mulai dari April hingga mencapai puncaknya di bulan Juli. Sementara itu, Musim kering terjadi dari September hingga Desember, di mana curah hujan menurun drastis dan mendekati nol pada akhir tahun. Pada tahun 2016 menunjukkan puncak curah hujan yang paling tinggi dibandingkan tahun lainnya.
